In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline

np.random.seed(2)

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

from keras.utils import to_categorical 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop,Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau


sns.set(style='white', context='notebook', palette='deep')

In [ ]:
train_data = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test_data = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

In [ ]:
train_data.head()

In [ ]:
y_train = train_data['label']
X = train_data.drop(labels='label',axis=1)
del train_data
X.head()

In [ ]:
g = sns.countplot(y_train)
y_train.value_counts()

In [ ]:
print(X.isnull().any().describe())
print(test_data.isnull().any().describe())

In [ ]:
X = X/255.0
test_data = test_data/255.0


In [ ]:
X = X.values.reshape(-1,28,28,1)
test_data = test_data.values.reshape(-1,28,28,1)


In [ ]:
y_train = to_categorical(y_train,num_classes=10)
random_seed=3

In [ ]:
X_train,X_val,y_train2,y_val = train_test_split(X,y_train,test_size=.1,random_state=random_seed)


In [ ]:
plt.imshow(X_train[0][:,:,0])

In [ ]:
model = Sequential()
model.add(Conv2D(filters=32,kernel_size=(5,5),padding = 'Same',activation='relu',input_shape=(28,28,1)))
model.add(Conv2D(filters=32,kernel_size=(5,5),padding = 'Same',activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters=64,kernel_size=(3,3),padding = 'Same',activation='relu'))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding = 'Same',activation='relu'))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Flatten())
model.add(Dense(256, activation ="relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation ="softmax"))

In [ ]:
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
LR_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)
epochs = 10
batch_size = 80

In [ ]:
model.fit(X_train, y_train2, batch_size = batch_size, epochs = epochs,
          validation_data = (X_val, y_val), verbose = 2)

In [ ]:
results = model.predict(test_data)
results = np.argmax(results,axis = 1)

results = pd.Series(results,name="Label")
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

submission.to_csv("submission.csv",index=False)